In [1]:
import requests
import sqlite3
import datetime
import os
from pathlib import Path
import pandas as pd
import requests
from IPython.display import display

In [2]:
# # To download all data between two dates

# import csv
# import requests
# import sqlite3

# # Define the date range
# start_date = 1995  # Starting year
# end_date = 2023    # Ending year
# user_agent = {"User-agent": "Mozilla/5.0"}

# # Establish a connection to the SQLite database
# conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
# cursor = conn.cursor()

# # Create the SEC_filings table if it doesn't exist
# cursor.execute('''CREATE TABLE IF NOT EXISTS SEC_filings (
#                     accession VARCHAR(255) PRIMARY KEY,
#                     cik INT,
#                     comnam VARCHAR(255),
#                     form VARCHAR(255),
#                     date DATE,
#                     url VARCHAR(255)
#                 )''')

# # Download and process the master.idx file for each year and quarter
# with open("master.idx", "wb") as f:
#     for year in range(start_date, end_date + 1):
#         for q in range(1, 5):
#             print(year, q)
#             content = requests.get(
#                 f"https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{q}/master.idx",
#                 headers=user_agent,
#             ).content
#             f.write(content)

#             # Write the downloaded content to the master.idx file
#             f.write(content)


In [4]:
# To import data for the last two quarters:

# Define the date range
current_year = datetime.datetime.now().year
current_quarter = (datetime.datetime.now().month - 1) // 3 + 1
previous_year = current_year if current_quarter > 1 else current_year - 1
previous_quarter = current_quarter - 1 if current_quarter > 1 else 4

user_agent = {"User-agent": "Mozilla/5.0"}

# Establish a connection to the SQLite database
conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db")
cursor = conn.cursor()

# Create the SEC_filings table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS SEC_filings (
                    accession VARCHAR(255) PRIMARY KEY,
                    cik INT,
                    comnam VARCHAR(255),
                    form VARCHAR(255),
                    date DATE,
                    url VARCHAR(255)
                )''')

# Download and process the master.idx file for the current and previous quarters
with open("master.idx", "wb") as f:
    print(f'Downloading Q{current_quarter} data of {current_year}')
    content_current = requests.get(
        f"https://www.sec.gov/Archives/edgar/full-index/{current_year}/QTR{current_quarter}/master.idx",
        headers=user_agent,
    ).content
    print(f'Downloading Q{previous_quarter} data of {previous_year}')
    content_previous = requests.get(
        f"https://www.sec.gov/Archives/edgar/full-index/{previous_year}/QTR{previous_quarter}/master.idx",
        headers=user_agent,
    ).content

    # Save the content of both quarters in the master.idx file
    f.write(content_current)
    f.write(content_previous)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [10]:
import csv

# Establish a connection to the SQLite database
conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db")
cursor = conn.cursor()

# Open and read the master.idx file
with open("master.idx", "r", encoding="latin1") as file:
    # Skip the header lines until the line containing "CIK|Company Name|Form Type|Date Filed|Filename"
    for _ in range(5):
        next(file)

    # Create the CSV reader
    reader = csv.reader(file, delimiter='|')

    # Find the first row after the header
    first_row = None
    for row in reader:
        if len(row) < 5:
            continue  # Skip rows with insufficient values

        if row[0] == "CIK":
            continue  # Skip the header row

        first_row = row
        break

    # # Print the first row of the reader
    # if first_row:
    #     print("First row:", first_row)

    # Batch processing variables
    batch_size = 1000  # Number of rows per batch
    batch_rows = []  # Rows to be inserted in a batch
    total_rows_added = 0  # Total number of rows added

    # Iterate over the remaining rows in master.idx
    for row in reader:
        if len(row) < 5:
            continue  # Skip rows with insufficient values

        if row[0] == "CIK":
            continue  # Skip the header row if encountered again

        cik = int(row[0])
        comnam = row[1]
        form = row[2]
        date = row[3]
        url = row[4]
        accession = url.split("/")[-1].split(".")[0]

        # Create a tuple representing the row data
        row_data = (accession, cik, comnam, form, date, url)
        batch_rows.append(row_data)

        # Check if the batch size is reached
        if len(batch_rows) >= batch_size:
            # Insert the batch of rows into the SEC_filings table, ignoring duplicates
            cursor.executemany("INSERT OR IGNORE INTO SEC_filings (accession, cik, comnam, form, date, url) VALUES (?, ?, ?, ?, ?, ?)",
                               batch_rows)
            num_rows_added = cursor.rowcount  # Get the number of rows added in this batch
            total_rows_added += num_rows_added  # Update the total row count
            # print(f"Rows added: {num_rows_added}")

            batch_rows = []  # Clear the batch rows

    # Insert any remaining rows in the last batch
    if batch_rows:
        cursor.executemany("INSERT OR IGNORE INTO SEC_filings (accession, cik, comnam, form, date, url) VALUES (?, ?, ?, ?, ?, ?)",
                           batch_rows)
        num_rows_added = cursor.rowcount  # Get the number of rows added in the last batch
        total_rows_added += num_rows_added  # Update the total row count

    print(f"Total rows added: {total_rows_added}")

# Commit the changes and close the connection
conn.commit()
conn.close()


Total rows added: 0


In [2]:
#V2:
import requests
import time
import os
import requests
from bs4 import BeautifulSoup

def doGet(url):
    s = requests.Session()
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Sec-CH-UA': 'Examplary Browser',
        'Sec-CH-UA-Mobile': '?0',
        'Sec-CH-UA-Platform': "Windows",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "authority": "www.sec.gov",
        "method": "GET",
        "path": "/Archives/edgar/data/59478/000120919121046268/0001209191-21-046268-index.htm",
        "scheme": "https",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip deflate br",
        "accept-language": "en-US,en;q=0.9,ar-LB;q=0.8,ar;q=0.7",
        "cache-control": "max-age=0"}


    r = None
    proxies = {
        'http': 'socks5://127.0.0.1:9050',
        'https': 'socks5://127.0.0.1:9050'
    }
    try:
        r = s.get(url, headers=headers)

        if r.status_code == 403:
            i = 1
            while r.status_code == 403 and i <= 10:
                print("Status code: " + str(r.status_code) + "  for : " + url)
                time.sleep(1)
                r = s.get(url, headers=headers, proxies = proxies)
                i += 1
        # print("Status code: " + str(r.status_code) + "  for : " + url)
        # r.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print("Http Error: " + errh + " for url: " + url)
        if r.get_status_code == 403:
            i = 1
            while r.status_code == 403 and i <= 10:
                time.sleep(i)
                r = s.get(url, headers=headers)
                i += 1
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting: " + str(errc) + "  for : " + url)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error: " + str(errt) + "  for : " + url)
    except requests.exceptions.RequestException as err:
        print("OOps: Something Else" + str(err) + "  for : " + url)

    return r

In [5]:
# Assign the desired values for filing and directory
filings = ["13D","13G","13F"]

for filing in filings:
    directory = f"data/{filing}"
    os.makedirs(directory, exist_ok=True)

    # # Get a list of all subfolders
    # subfolders = sorted([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))], reverse = True)
    
    # to get the years of the last 2 quarters updated - otherwise use the subfolders code above to update the whole database
    subfolders = sorted([str(current_year),str(previous_year)], reverse = True)
    subfolders = [folder for folder in subfolders if folder] #remove empty values

    # Iterate over the subfolders
    for subfolder in subfolders:
        # print(f'Looking at {filing} filings from year {subfolder}')
        subdirectory = os.path.join(directory, subfolder)
        # processed_subdirectory = os.path.join(processed_directory, subfolder)
        # # Create the processed subdirectory if it doesn't exist
        # os.makedirs(processed_subdirectory, exist_ok=True)
        
        # Create a list of filingIDs from the data directory
        filingIDs = []
        for filename in os.listdir(subdirectory):
            if filename.endswith(".txt"):
                filingID = os.path.splitext(filename)[0]
                filingIDs.append(filingID)
        # print(f'Filings already downloaded in year {subfolder}: {len(filingIDs)}')


        # Connect to the SQLite database and find already parsed filings
        conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 
        cursor = conn.cursor()
        # Get a list of 13G filingIDs from the SQL table
        select_query = f"SELECT accession FROM SEC_{filing}_filings WHERE date LIKE '{subfolder}%'"
        cursor.execute(select_query)

        # Fetch all rows and extract the accessions
        sql_filingIDs = [row[0] for row in cursor.fetchall()]

        filings_to_download = list(set(sql_filingIDs) - set(filingIDs))
        filings_to_delete = list(set(filingIDs) - set(sql_filingIDs))

        # print(f'Filings to be downloaded in year {subfolder}: {len(filings_to_download)}')
        print(f'Year {subfolder} - Downloaded {filing} Filings: {len(filingIDs)}. Remaining: {len(filings_to_download)}')

        # Create a comma-separated string of filings_to_download for the accessions
        filings_to_download_str = ', '.join(['?' for _ in filings_to_download])

        # Prepare the SELECT query
        select_query = f"SELECT accession, cik, comnam, form, date, url FROM SEC_{filing}_filings WHERE accession IN ({filings_to_download_str})"
        # Execute the query with the list of accessions
        cursor.execute(select_query, filings_to_download)

        # Fetch all the rows returned by the query
        rows = cursor.fetchall()

        # Get the column names from the cursor description
        columns = [column[0] for column in cursor.description]

        # Convert the rows into a list of dictionaries
        rows = [dict(zip(columns, row)) for row in rows]

        url_fails = []
        previous_year = ""
        deleted_count = 0

        for n, row in enumerate(rows):
            if n % 1000 == 0:
                print('Parsing batch number '+str())
            cik = row["cik"]
            date = row["date"].strip()
            year = row["date"].split("-")[0].strip()
            # month = row["date"].split("-")[1].strip()
            url = row["url"].strip()
            accession = url.split(".")[0].split("/")[-1]
            Path(f"./{directory}/{year}").mkdir(parents=True, exist_ok=True)
            file_path = f"./{directory}/{year}/{accession}.txt"
            # processed_file_path = f"./{processed_directory}/{year}/{accession}.txt"

            if year != previous_year:
                print(f'Year: {year}')
                previous_year = year

            if os.path.exists(file_path):# or os.path.exists(processed_file_path):
                if os.path.getsize(file_path) == 0:
                    os.remove(file_path)
                    deleted_count += 1
                    try:
                        document_url = f"https://www.sec.gov/Archives/{url}"
                        # print(f'Document URL: {document_url}')
                        document_response = doGet(document_url)
                        if document_response.content:
                            with open(file_path, "wb") as file:
                                file.write(document_response.content)
                                pass
                        else:
                            display(f"{cik}, {accession}, {date} failed to download - Empty content")
                            url_fails.append(row)
                            pass
                    except:
                        display(f"{cik}, {accession}, {date} failed to download")
                        url_fails.append(row)
                        pass
                    pass

                else:
                    with open(file_path, 'r') as f:
                        try:
                            first_line = f.readline()
                        except:
                            print(file_path, Exception)
                            pass      
                        
                    if first_line.startswith('<!DOCTYPE html'):
                        os.remove(file_path)
                        deleted_count += 1

                        try:
                            document_url = f"https://www.sec.gov/Archives/{url}"
                            # print(f'Document URL: {document_url}')
                            document_response = doGet(document_url)
                            if document_response.content:
                                with open(file_path, "wb") as file:
                                    file.write(document_response.content)
                                    pass
                            else:
                                display(f"{cik}, {accession}, {date} failed to download - Empty content")
                                url_fails.append(row)
                                pass
                        except:
                            display(f"{cik}, {accession}, {date} failed to download")
                            url_fails.append(row)
                            pass
                        pass

            else:
                try:
                    document_url = f"https://www.sec.gov/Archives/{url}"
                    # print(f'Document URL: {document_url}')
                    document_response = doGet(document_url)
                    if document_response.content:
                        with open(file_path, "wb") as file:
                            file.write(document_response.content)
                    else:
                        display(f"{cik}, {accession}, {date} failed to download - Empty content")
                        url_fails.append(row)
                        pass

                except:
                    display(f"{cik}, {accession}, {date} failed to download")
                    url_fails.append(row)



        print(f"Number of files deleted: {deleted_count}")

        # Close the connection
        conn.close()



Year 2023 - Downloaded 13D Filings: 3097. Remaining: 21
0
Year: 2023
Number of files deleted: 0
Year 2023 - Downloaded 13D Filings: 3118. Remaining: 0
Number of files deleted: 0
Year 2023 - Downloaded 13G Filings: 21972. Remaining: 6
0
Year: 2023
Number of files deleted: 0
Year 2023 - Downloaded 13F Filings: 15111. Remaining: 123
0
Year: 2023
Number of files deleted: 0
Year 2023 - Downloaded 13F Filings: 15234. Remaining: 0
Number of files deleted: 0
